#### Bias only model 
 - Write down code for the gradient equations
 - Use gradient descent library to optimize the model

In [5]:
import gzip
from collections import defaultdict
import random
import numpy
import scipy.optimize

In [6]:
path = "/Users/cnogueira/Documents/Development/Notebooks/study/week2/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"

In [7]:
f = gzip.open(path, 'rt', encoding="utf8")
header = f.readline()

In [8]:
header = header.strip().split('\t')

In [9]:
dataset = []
for line in f: 
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)

In [10]:
# Here i am creating two data structures:  
#For each user, which itens did they consume (users per item), and
#For each items, which users consumed that item (item per user)
usersPerItem = defaultdict(set) #Ui
itemsPerUser = defaultdict(set) #Iu
itemNames = {}
for d in dataset:
    user, item = d['customer_id'], d['product_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    itemNames[item] = d['product_title']

In [13]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
for d in dataset:
    user, item = d['customer_id'], d['product_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [14]:
N = len(dataset)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

In [15]:
# This is just to see if the algorithm is better to just predict the mean in terms of mean square error all the time
ratingMean = sum([d['star_rating'] for d in dataset])/len(dataset)

### Fitting
    - Alpha and Betau/Betai (userbiases) are the parameters I will fit.  

In [16]:
# Initial value of alpha is the mean reating to help the algo to convert qickly
alpha = ratingMean

In [18]:
# Initial valie of betai and betau is zerp
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [20]:
# The prediction function is just the bias only model
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

In [ ]:
# Unpack function
# The gradient descent library expects a single vector of parameters (theta) 
# On this function I am unpacking back to produce alpha and betau and betai

def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0] # Alpha is the first position on the vector
    userBiases = dict(zip(users, theta[1:nUsers+1])) 
    # Start on position 1 and go until the rest of the users onwards to the rest of that vector
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [ ]:
# Cost function is also required by the library (is written on the notes)
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [predictions(d['customer_id'], d['product_id']) for d in dataset]
    